**深度学习发展历史** 
1940年代：首次提出神经元的结构，但权重是不可学的。 50-60年代：提出权重学习理论，神经元结构趋于完善，开启了神经网络的第一个黄金时代。 1969年：提出异或问题（人们惊讶的发现神经网络模型连简单的异或问题也无法解决，对其的期望从云端跌落到谷底），神经网络模型进入了被束之高阁的黑暗时代。 1986年：新提出的多层神经网络解决了异或问题，但随着90年代后理论更完备并且实践效果更好的SVM等机器学习模型的兴起，神经网络并未得到重视。 2010年左右：深度学习进入真正兴起时期。随着神经网络模型改进的技术在语音和计算机视觉任务上大放异彩，也逐渐被证明在更多的任务，如自然语言处理以及海量数据的任务上更加有效。至此，神经网络模型重新焕发生机，并有了一个更加响亮的名字：深度学习。

**人工智能、机器学习、深度学习的区别与联系** 
人工智能、机器学习和深度学习覆盖的技术范畴是逐层递减的。人工智能是最宽泛的概念。机器学习是当前比较有效的一种实现人工智能的方式。深度学习是机器学习算法中最热门的一个分支，近些年取得了显著的进展，并替代了大多数传统机器学习算法

**神经元、单层感知机、多层感知机**
神经元：神经元接收来自 nn 个其他神经元传递过来的输入信号 xx，这些输入信号通过带权重 ww 的连接(connection)进行传递，神经元接收到的总输入值 ∑ni=1wixi∑i=1nwixi 将与神经元的阈值 θθ 进行比较，然后通过“激活函数”(activation function) ff 处理产生神经元的输出 y=f(∑ni=1wixi−θ)y=f(∑i=1nwixi−θ)。

单层感知机：单层感知机是二分类的线性分类模型，输入是被感知数据集的特征向量，输出时数据集的类别{+1,-1}。单层感知机的函数近似非常有限，其决策边界必须是一个超平面，严格要求数据是线性可分的。

多层感知机：多层感知机（MLP，Multilayer Perceptron）也叫人工神经网络（ANN，Artificial Neural Network），除了输入输出层，它中间可以有多个隐层，最简单的MLP只含一个隐层，即三层的结构

**前向传播与反向传播**
前向传播：
![](https://ai-studio-static-online.cdn.bcebos.com/cbc7c4eb65ba444c8042b1d8f8d5911f796c4eb4032e46e29eafd722f8c94689)
反向传播：


In [16]:
#加载飞桨、Numpy和相关类库
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import numpy as np
import os
import random

In [17]:
def load_data():
    # 从文件导入数据
    datafile = './work/housing.data'
    data = np.fromfile(datafile, sep=' ', dtype=np.float32)

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算train数据集的最大值，最小值，平均值
    maximums, minimums, avgs = training_data.max(axis=0), training_data.min(axis=0), \
                                 training_data.sum(axis=0) / training_data.shape[0]
    
    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
    global avg_values
    max_values = maximums
    min_values = minimums
    avg_values = avgs

    # 对数据进行归一化处理
    for i in range(feature_num):
        data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data


In [18]:
class Regressor(paddle.nn.Layer):

    # self代表类的实例自身
    def __init__(self):
        # 初始化父类中的一些参数
        super(Regressor, self).__init__()
        
        # 定义一层全连接层，输入维度是13，输出维度是1
        self.fc1 = Linear(in_features=13, out_features=5)
        self.fc2=Linear(in_features=5,out_features=1)
    
    # 网络的前向计算
    def forward(self, inputs):
        x1 = self.fc1(inputs)
        x=self.fc2(x1)
        return x

In [19]:
# 声明定义好的线性回归模型
model = Regressor()
# 开启模型训练模式
model.train()
# 加载数据
training_data, test_data = load_data()
# 定义优化算法，使用随机梯度下降SGD
# 学习率设置为0.01
opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())

In [20]:
EPOCH_NUM = 10   # 设置外层循环次数
BATCH_SIZE = 10  # 设置batch大小

# 定义外层循环
for epoch_id in range(EPOCH_NUM):
    # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
    np.random.shuffle(training_data)
    # 将训练数据进行拆分，每个batch包含10条数据
    mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
    # 定义内层循环
    for iter_id, mini_batch in enumerate(mini_batches):
        x = np.array(mini_batch[:, :-1]) # 获得当前批次训练数据
        y = np.array(mini_batch[:, -1:]) # 获得当前批次训练标签（真实房价）
        # 将numpy数据转为飞桨动态图tensor形式
        house_features = paddle.to_tensor(x)
        prices = paddle.to_tensor(y)
        
        # 前向计算
        predicts = model(house_features)
        
        # 计算损失
        loss = F.square_error_cost(predicts, label=prices)
        avg_loss = paddle.mean(loss)
        if iter_id%20==0:
            print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
        
        # 反向传播
        avg_loss.backward()
        # 最小化loss,更新参数
        opt.step()
        # 清除梯度
        opt.clear_grad()


epoch: 0, iter: 0, loss is: [0.02308946]
epoch: 0, iter: 20, loss is: [0.11322957]
epoch: 0, iter: 40, loss is: [0.18816632]
epoch: 1, iter: 0, loss is: [0.04931533]
epoch: 1, iter: 20, loss is: [0.06234761]
epoch: 1, iter: 40, loss is: [0.00816777]
epoch: 2, iter: 0, loss is: [0.07017995]
epoch: 2, iter: 20, loss is: [0.0247428]
epoch: 2, iter: 40, loss is: [0.04213668]
epoch: 3, iter: 0, loss is: [0.02717801]
epoch: 3, iter: 20, loss is: [0.06972653]
epoch: 3, iter: 40, loss is: [0.01825646]
epoch: 4, iter: 0, loss is: [0.03001045]
epoch: 4, iter: 20, loss is: [0.03436211]
epoch: 4, iter: 40, loss is: [0.00149416]
epoch: 5, iter: 0, loss is: [0.05370724]
epoch: 5, iter: 20, loss is: [0.00968384]
epoch: 5, iter: 40, loss is: [0.0105168]
epoch: 6, iter: 0, loss is: [0.02274525]
epoch: 6, iter: 20, loss is: [0.02020337]
epoch: 6, iter: 40, loss is: [0.03051201]
epoch: 7, iter: 0, loss is: [0.05805669]
epoch: 7, iter: 20, loss is: [0.02365099]
epoch: 7, iter: 40, loss is: [0.01983729]
ep

这个实现过程令人惊喜，前向计算、计算损失和反向传播梯度，每个操作居然只有1-2行代码即可实现！框架内部帮我们自动实现反向梯度计算和参数更新的过程，我们再也不用一点点的实现模型训练的细节，这就是使用飞桨框架的威力！

In [21]:
# 保存模型参数，文件名为LR_model.pdparams
paddle.save(model.state_dict(), 'LR_model.pdparams')
print("模型保存成功，模型参数保存在LR_model.pdparams中")

模型保存成功，模型参数保存在LR_model.pdparams中


In [22]:
def load_one_example():
    # 从上边已加载的测试集中，随机选择一条作为测试数据
    idx = np.random.randint(0, test_data.shape[0])
    idx = -10
    one_data, label = test_data[idx, :-1], test_data[idx, -1]
    # 修改该条数据shape为[1,13]
    one_data =  one_data.reshape([1,-1])

    return one_data, label


In [23]:
# 参数为保存模型参数的文件地址
model_dict = paddle.load('LR_model.pdparams')
model.load_dict(model_dict)
model.eval()

# 参数为数据集的文件地址
one_data, label = load_one_example()
# 将数据转为动态图的variable格式 
one_data = paddle.to_tensor(one_data)
predict = model(one_data)

# 对结果做反归一化处理
predict = predict * (max_values[-1] - min_values[-1]) + avg_values[-1]
# 对label数据做反归一化处理
label = label * (max_values[-1] - min_values[-1]) + avg_values[-1]

print("Inference result is {}, the corresponding label is {}".format(predict.numpy(), label))

Inference result is [[19.5219]], the corresponding label is 19.700000762939453
